In [69]:
import pandas as pd

In [83]:
train = pd.read_csv("Corona_NLP_train.csv", index_col = 0, encoding='latin-1')
test = pd.read_csv("Corona_NLP_test.csv", index_col = 0, encoding='latin-1')

In [84]:
#Map encodings

encoded_dict = {'Extremely Negative':0, 'Negative':0, 'Neutral':1, 'Positive':2, 'Extremely Positive':2}

train['Sentiment'] = train.Sentiment.map(encoded_dict)
test['Sentiment'] = test.Sentiment.map(encoded_dict)

In [85]:
#Label to categorical representation

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train.Sentiment)
y_test = to_categorical(test.Sentiment)

In [ ]:
#Define tokenizer/bert

import transformers
from transformers import AutoTokenizer,TFBertModel

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

In [87]:
#Tokenize the input

X_train = tokenizer(
    text = train.OriginalTweet.tolist(),
    add_special_tokens = True,
    max_length = 70,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
X_test = tokenizer(
    text = test.OriginalTweet.tolist(),
    add_special_tokens = True,
    max_length = 70,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [88]:
input_ids = X_train['input_ids']
attention_mask = X_train['attention_mask']

In [92]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.layers import Input, Dense

max_len = 70
input_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_ids")
input_mask = Input(shape = (max_len,), dtype = tf.int32, name = "attention_mask")
embeddings = bert(input_ids, attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation = 'relu')(out)
y = Dense(3, activation = 'sigmoid')(out)          #Change for number of labels
model = tf.keras.Model(inputs = [input_ids, input_mask], outputs = y)
model.layers[2].trainable = True

#Compile the model
model.compile(
    optimizer = 'adam',
    loss = "categorical_crossentropy")

In [ ]:
#Train model

train_history = model.fit(
    x = {'input_ids':X_train['input_ids'], 'attention_mask': X_train['attention_mask']},
    y = y_train,
    validation_data = (
    {'input_ids':X_test['input_ids'], 'attention_mask': X_test['attention_mask']}, y_test),
    epochs = 1,
    batch_size = 36
)

In [94]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

119/119 [==============================] - 105s 869ms/step


In [95]:

y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = test.Sentiment

from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.43      1.00      0.60      1633
           1       0.00      0.00      0.00       619
           2       0.00      0.00      0.00      1546

    accuracy                           0.43      3798
   macro avg       0.14      0.33      0.20      3798
weighted avg       0.18      0.43      0.26      3798



/swst/apps/anaconda3/2021.05_gcc-8.5.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/swst/apps/anaconda3/2021.05_gcc-8.5.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/swst/apps/anaconda3/2021.05_gcc-8.5.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta